###In this notebook: Extract some further details (e.g. ratings and scores) from Charity Navigator (local) pages. 
###Create a unified dictionary with all information on charities: Name, URL, Location, Mission Statement, Ratings

In [135]:
import os
import pickle

from bs4 import BeautifulSoup
from collections import defaultdict

In [62]:
directory = "..."
alerts_charities2exclude = []
orgID_ScoreStars = {}

for filename in os.listdir(directory):
    if filename.endswith(".html"):
        
        orgID = filename.split(".")[0]
              
        filepath = os.path.join(directory, filename)
        
        with open(filepath, "r") as f:
            data = f.read()
            soup = BeautifulSoup(data)

        
        try:
            #Already collected below, not necessary to collect again
            #charityName = soup.find("h1", {"class": "charityname"})
            #tagLine = soup.find("h2", {"class": "tagline"})    

            rating_wrapper = soup.find("div", {"class": "rating-wrapper"})
            summaryBox = rating_wrapper.find("div", {"class": "summaryBox"})

            score = summaryBox.find("td", {"align": "center"})
            stars = soup.find("svg", {"class": "stars"})
        
            orgID_ScoreStars[orgID] = [score.text, stars.text]
            
        except:
            #Exception occurs when Charity has a warning and missing fields like mission statement
            alerts_charities2exclude.append(charityNumber)
            with open("exception.txt", "a") as wf:
                wf.write(str(charityNumber) + " returned exception" + "\n")
            
    else:
        pass


###Load previously saved dictionaries from pickles

In [ ]:
with open('orgID_missionDict.pickle', 'rb') as handle1:
    orgID_missionDict = pickle.load(handle1)
    
with open('orgID_websiteDict.pickle', 'rb') as handle2:
    orgID_websiteDict = pickle.load(handle2)
    
with open('orgID_Name.pickle', 'rb') as handle3:
    orgID_Name = pickle.load(handle3)
    
with open('orgID_State.pickle', 'wb') as handle:
    pickle.dump(orgID_State, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [139]:
#Combine all dictionaries
all_Combined = defaultdict(list)

for d in (orgID_Name, orgID_State,orgID_websiteDict,orgID_missionDict,orgID_ScoreStars):
    for key, value in d.items():
        all_Combined[key].append(value)
        
all_Combined_Dict = dict(all_Combined)

In [211]:
#Remove entries that are missing mission statement or scores
keys2remove = []

for key, value in all_Combined_Dict.items():
    if len(value) < 5:
        keys2remove.append(key)

In [ ]:
for k in keys2remove:
    del all_Combined_Dict[k]

In [217]:
with open('all_Combined_Cleaned.pickle', 'wb') as handle:
    pickle.dump(all_Combined_Dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [234]:
#Remove entries that have a less than 3 star rating
keys2remove_4ratings = []
for k,v in all_Combined_Dict.items():
    if float(v[4][0]) < 80.0:
        keys2remove_4ratings.append(k)

In [236]:
for k in keys2remove_4ratings:
    del all_Combined_Dict[k]

In [238]:
with open('all_Combined_Cleaned_Ratings_Selection.pickle', 'wb') as handle:
    pickle.dump(all_Combined_Dict, handle, protocol=pickle.HIGHEST_PROTOCOL)